In [1]:
import pandas as pd
import numpy as np
import skfda

from skfda.preprocessing.dim_reduction import FPCA
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR

path = './dataset/Electricity_dataset.csv'
df =  pd.read_csv(path, index_col=0)

In [2]:
days = 73
p = 1
n_components = 5

test_df = df.groupby('unique_id').tail(24 * days)
train_df = df.groupby('unique_id').apply(lambda group: group.iloc[:-24 * days]).reset_index(drop=True)

test_df = test_df.pivot(index="ds", columns="unique_id", values="y").reset_index()
train_df = train_df.pivot(index="ds", columns="unique_id", values="y").reset_index()

# order of the list: 'AP-AP', 'DOM-DOM', 'JC-JC', 'PN-PN', 'RTO-RTO'
train_array = train_df.drop(columns='ds').T.values.reshape(5, -1,24)
test_array = test_df.drop(columns='ds').T.values.reshape(5, -1,24)

inps = np.concatenate((train_array[:, -1 * p:, :], test_array), axis = 1) #concat all inputs
#inputs = np.array([inps[:, i: i + p,:] for i in range(inps.shape[1] - p)]) # divide them into pairs by p


/tmp/ipykernel_62538/1782554617.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_df = df.groupby('unique_id').apply(lambda group: group.iloc[:-24 * days]).reset_index(drop=True)


In [3]:
data_matrix = train_array  # n days × 24 time points
grid_points = np.linspace(0, 1, 24)  # 24 time points over [0,1]

# Define labels for functional data objects
labels = ["ap", "dom", "jc", "pn", "rto"]

array_objects = {label: data_matrix[i] for i, label in enumerate(labels)}
array_inputs = {label: inps[i] for i, label in enumerate(labels)}

fd_objects = {label: skfda.FDataGrid(data_matrix[i], grid_points) for i, label in enumerate(labels)}
fd_inputs = {label: skfda.FDataGrid(inps[i], grid_points) for i, label in enumerate(labels)}


In [4]:
from skfda.representation.basis import BSplineBasis
from skfda.preprocessing.smoothing import BasisSmoother


def PCAVAR_fit_predict(train_fd, input_fd, n_components=5, p=1, steps=1):
    
    # Apply FPCA to extract empirical basis functions
    pca = PCA(n_components=n_components)
    pca.fit(train_fd)

    # Transform the original data into PCA scores
    scores = pca.transform(train_fd)
    scores_df = pd.DataFrame(scores, columns=[f'PC{i+1}' for i in range(scores.shape[1])])

    # Fit VAR(p) model
    var_model = VAR(scores)
    var_result = var_model.fit(maxlags=p)  # WITH intercept

    # Convert inputs FD into PCA scores
    input_scores = pca.transform(input_fd)      
    input_scores_array = np.array([input_scores[i: i + p, :] for i in range(inps.shape[1] - p)])

    preds = []
    
    for inputs in input_scores_array:

        forecast = var_result.forecast(inputs, steps=steps)
        reconstructed_fd = pca.inverse_transform(forecast)
        
        preds.append(reconstructed_fd)

    return np.array(preds).squeeze(), pca


def ARH_fit_predict(train_fd, input_fd, n_components=5, p=1, steps=1):
    
    basis = BSplineBasis(n_basis=12)  
    smoother = BasisSmoother(basis)
    fd_smooth = smoother.fit_transform(train_fd)
    
    fpca = FPCA(n_components=n_components)
    fpca.fit(fd_smooth)

    # Transform the original data into FPCA scores
    scores = fpca.transform(fd_smooth)
    scores_df = pd.DataFrame(scores, columns=[f'PC{i+1}' for i in range(scores.shape[1])])

    var_model = VAR(scores)
    var_result = var_model.fit(maxlags=p)  # WITH intercept

    # Convert inputs FD into FPCA scores
    input_scores = fpca.transform(input_fd)  
    input_scores_array = np.array([input_scores[i: i + p,:] for i in range(inps.shape[1] - p)])
    
    
    preds = []
    
    for inputs in input_scores_array:
    
        forecast = var_result.forecast(inputs, steps=steps)
        reconstructed_fd = fpca.inverse_transform(forecast)
        
        preds.append(reconstructed_fd.data_matrix[0])

    return np.array(preds).squeeze(), fpca



In [5]:
# ARH model predictions
ARH_preds_ap, fpca_ap     = ARH_fit_predict(fd_objects["ap"], fd_inputs["ap"], n_components=n_components, p=p, steps=1)
ARH_preds_dom, fpca_dom   = ARH_fit_predict(fd_objects["dom"], fd_inputs["dom"], n_components=n_components, p=p, steps=1)
ARH_preds_jc, fpca_jc     = ARH_fit_predict(fd_objects["jc"], fd_inputs["jc"], n_components=n_components, p=p, steps=1)
ARH_preds_pn, fpca_pn     = ARH_fit_predict(fd_objects["pn"], fd_inputs["pn"], n_components=n_components, p=p, steps=1)
ARH_preds_rto, fpca_rto   = ARH_fit_predict(fd_objects["rto"], fd_inputs["rto"], n_components=n_components, p=p, steps=1)

# PCA-VAR model predictions
PCAVAR_preds_ap, pca_ap     = PCAVAR_fit_predict(array_objects["ap"], array_inputs["ap"], n_components=n_components, p=p, steps=1)
PCAVAR_preds_dom, pca_dom   = PCAVAR_fit_predict(array_objects["dom"], array_inputs["dom"], n_components=n_components, p=p, steps=1)
PCAVAR_preds_jc, pca_jc     = PCAVAR_fit_predict(array_objects["jc"], array_inputs["jc"], n_components=n_components, p=p, steps=1)
PCAVAR_preds_pn, pca_pn     = PCAVAR_fit_predict(array_objects["pn"], array_inputs["pn"], n_components=n_components, p=p, steps=1)
PCAVAR_preds_rto, pca_rto   = PCAVAR_fit_predict(array_objects["rto"], array_inputs["rto"], n_components=n_components, p=p, steps=1)


In [6]:
print(fpca_ap.explained_variance_ratio_.sum())
print(fpca_dom.explained_variance_ratio_.sum())
print(fpca_jc.explained_variance_ratio_.sum())
print(fpca_pn.explained_variance_ratio_.sum())
print(fpca_rto.explained_variance_ratio_.sum())


print("\n", pca_ap.explained_variance_ratio_.sum())
print(pca_dom.explained_variance_ratio_.sum())
print(pca_jc.explained_variance_ratio_.sum())
print(pca_pn.explained_variance_ratio_.sum())
print(pca_rto.explained_variance_ratio_.sum())

0.9979021069439495
0.9976546520274417
0.9973385982611992
0.9956178377360317
0.9984475285312647

 0.9971488485915895
0.9969960086179049
0.996307120215542
0.9933916456646801
0.9980166526966494


In [7]:
def calculate_errors(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    mae = np.mean(np.abs(y_true - y_pred))
    mse = np.mean((y_true - y_pred) ** 2)
    
    return {"MAE": mae, "MSE": mse}


In [8]:
# order of the list: 'AP-AP', 'DOM-DOM', 'JC-JC', 'PN-PN', 'RTO-RTO'
print("ARH metrics:\n")
print(calculate_errors(test_array[0], ARH_preds_ap))
print(calculate_errors(test_array[1], ARH_preds_dom))
print(calculate_errors(test_array[2], ARH_preds_jc))
print(calculate_errors(test_array[3], ARH_preds_pn))
print(calculate_errors(test_array[4], ARH_preds_rto))


# order of the list: 'AP-AP', 'DOM-DOM', 'JC-JC', 'PN-PN', 'RTO-RTO'
print("\nPCA + VAR metrics:\n")
print(calculate_errors(test_array[0], PCAVAR_preds_ap))
print(calculate_errors(test_array[1], PCAVAR_preds_dom))
print(calculate_errors(test_array[2], PCAVAR_preds_jc))
print(calculate_errors(test_array[3], PCAVAR_preds_pn))
print(calculate_errors(test_array[4], PCAVAR_preds_rto))

ARH metrics:

{'MAE': 242.72823471639933, 'MSE': 116187.86237483878}
{'MAE': 622.7588232875279, 'MSE': 684669.0324304459}
{'MAE': 219.67997002952526, 'MSE': 112751.31774540889}
{'MAE': 79.24471565831568, 'MSE': 11460.156374459657}
{'MAE': 3630.277702128358, 'MSE': 22705331.380642965}

PCA + VAR metrics:

{'MAE': 240.99230654137767, 'MSE': 116297.62391418676}
{'MAE': 612.0931060470923, 'MSE': 667872.3201745803}
{'MAE': 217.91814946264816, 'MSE': 112294.48098146821}
{'MAE': 78.72829089716795, 'MSE': 11430.801145007634}
{'MAE': 3598.3232506665195, 'MSE': 22660110.583822284}


In [9]:

arh_results = pd.DataFrame({
    "AP-AP": ARH_preds_ap.ravel(),
    "DOM-DOM": ARH_preds_dom.ravel(),
    "JC-JC": ARH_preds_jc.ravel(),
    "PN-PN": ARH_preds_pn.ravel(),
    "RTO-RTO": ARH_preds_rto.ravel()
})

pca_results = pd.DataFrame({
    "AP-AP": PCAVAR_preds_ap.ravel(),
    "DOM-DOM": PCAVAR_preds_dom.ravel(),
    "JC-JC": PCAVAR_preds_jc.ravel(),
    "PN-PN": PCAVAR_preds_pn.ravel(),
    "RTO-RTO": PCAVAR_preds_rto.ravel()
})

arh_results.to_csv("preds/ARH(1) predictions/arh1_all_73days.csv", index=False)
pca_results.to_csv("preds/PCA(1) predictions/pca_all_73days.csv", index=False)
